In [1]:
import nest_asyncio

nest_asyncio.apply()
import os
import json

with open("secrets.json") as f:
    secrets = json.load(f)
    for env_var_name in secrets:
        os.environ[env_var_name] = secrets[env_var_name]

In [2]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["Origin_of_Species.pdf"]).load_data()

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

NameError: name 'documents' is not defined

In [3]:
# from llama_index.core import Settings
# from llama_index.llms.ollama import Ollama
# from llama_index.embeddings.ollama import OllamaEmbedding
# 
# Settings.llm = Ollama(model="qwen:0.5b", request_timeout=60.0)
# 
# ollama_embedding = OllamaEmbedding(
#     model_name="qwen:0.5b",
#     base_url="http://localhost:11434",
#     ollama_additional_kwargs={"mirostat": 0},
# )
# Settings.embed_model = ollama_embedding

# running local model above is super slow. 127 times slower to be precise
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")


In [5]:
completion_result = Settings.llm.complete("halo")

In [6]:
completion_result.text

'Hello! How can I assist you today?'

In [4]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

NameError: name 'nodes' is not defined

In [6]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

NameError: name 'summary_index' is not defined

In [9]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context"
    ),
)

NameError: name 'summary_query_engine' is not defined

In [7]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

NameError: name 'summary_tool' is not defined

In [8]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

NameError: name 'query_engine' is not defined